#Task Definition

##The task is to build a Support Vector Machine for classification. You will make
use of a library routine to solve the optimization problem which emerges in the
dual formulation of the support vector machine. You need to write code for
structuring the data so that the library routine can find the maximal-margin
solution, and code for transforming this solution into a classifier which can
process new data.

In [181]:
# import all necessary libraries
!pip install numpy
!pip install matplotlib
!pip install scipy
import numpy as np
import random
import matplotlib.pyplot as plt
from scipy.optimize import minimize


**Generate kernels required (linear kernel function, polynomial kernel function and radical basis function)

In [ ]:
# Linear kernel function
def linear_kernel(x, y):
    return np.dot(x, y)

# Polynomial kernel function (degree=2)
def polynomial_kernel(x, y, p=2):
    return (np.dot(x, y) + 1) ** p

# Radial Basis Function (RBF) kernel
def rbf_kernel(x, y, sigma=1):
    return np.exp(-np.linalg.norm(x - y) ** 2 / (2 * sigma ** 2))

**Define Objective function and equality constraints function

In [ ]:
# Objective function to minimize (Equation 4)
def objective(alpha):
    return 0.5 * np.dot(alpha, np.dot(alpha * targets, P)) - np.sum(alpha)

In [ ]:
# Equality constraint function (sum of alpha_i * t_i = 0)
def zerofun(alpha):
    return np.dot(alpha, targets)

**Generate Data and visualize

In [ ]:
# Generate training data
np.random.seed(100)
classA = np.concatenate((np.random.randn(10, 2) * 0.2 + [1.5, 0.5],
                         np.random.randn(10, 2) * 0.2 + [-1.5, 0.5]))

classB = np.random.randn(20, 2) * 0.2 + [0.0, -0.5]

inputs = np.concatenate((classA, classB))
targets = np.concatenate((np.ones(classA.shape[0]), -np.ones(classB.shape[0])))


# Shuffle data
N = inputs.shape[0]
permute = list(range(N))
random.shuffle(permute)
inputs = inputs[permute, :]
targets = targets[permute]

In [ ]:
# Visualizing the data before applying SVM
plt.figure()
plt.scatter(classA[:, 0], classA[:, 1], color='blue', label='Class A')
plt.scatter(classB[:, 0], classB[:, 1], color='red', label='Class B')
plt.xlabel("X-axis")
plt.ylabel("Y-axis")
plt.legend()
plt.title("Primary generated data Before Applying SVM")
plt.savefig("Primarydata.pdf")
plt.show()